In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
data_time = '20180920'

In [3]:
ENWIKI_ALI_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if file.endswith("red.csv.gz"):
        ENWIKI_ALI_FNMS.append(file)  

In [4]:
EN_ID_NAME = "enwiki-" + data_time + "-id_name.csv"  

In [5]:
len(ENWIKI_ALI_FNMS)

55

In [6]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [7]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [9]:
df_en_id_name = pd.read_csv(PATH_TO_DATA+EN_ID_NAME, encoding='utf-8', sep="^")
df_en_id_name.head()

,id,title
0,6197600,Flying Tigers (film)
1,6197604,Mariano Andreu
2,6197607,David Prain
3,6197617,Rut Brandt
4,6197621,Gregory C. Coleman


In [10]:
df_id_alias_title_alias = None
df_id_alias_id_orig = None
df_id_orig_title_alias = None
for fn in ENWIKI_ALI_FNMS:
    fn = PATH_TO_DATA+fn
    print(fn)
    fn_new = unpack(fn)
    df_aliases = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    df_aliases = pd.merge(df_aliases, df_en_id_name, how='left', left_on=['redirect'], right_on=['title'])    
    df_aliases = df_aliases[['id_x', 'id_y', 'title_x', 'redirect']]
    df_aliases.columns = ['id_alias', 'id_orig', 'title_alias', 'title_orig']    
    
    df_id_alias_title_alias_tmp = df_aliases[['id_alias', 'title_alias']]
    if df_id_alias_title_alias is not None:
        df_id_alias_title_alias = df_id_alias_title_alias.append(df_id_alias_title_alias_tmp)        
    else:
        df_id_alias_title_alias = df_id_alias_title_alias_tmp
    print("df_id_alias_title_alias size: {}".format(df_id_alias_title_alias.shape))
    
    df_id_alias_id_orig_tmp = df_aliases[['id_alias', 'id_orig']]
    if df_id_alias_id_orig is not None:
        df_id_alias_id_orig = df_id_alias_id_orig.append(df_id_alias_id_orig_tmp)        
    else:
        df_id_alias_id_orig = df_id_alias_id_orig_tmp
    print("df_id_alias_id_orig size: {}".format(df_id_alias_id_orig.shape))
    
    df_id_orig_title_alias_tmp = df_aliases[['id_orig', 'title_alias']]
    if df_id_orig_title_alias is not None:
        df_id_orig_title_alias = df_id_orig_title_alias.append(df_id_orig_title_alias_tmp)        
    else:
        df_id_orig_title_alias = df_id_orig_title_alias_tmp
    print("df_id_orig_title_alias size: {}".format(df_id_orig_title_alias.shape))
    os.remove(fn_new)

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current27.xml-p56163462p57663462.bz2_red.csv.gz
df_id_alias_title_alias size: (201075, 2)
df_id_alias_id_orig size: (201075, 2)
df_id_orig_title_alias size: (201075, 2)
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current18.xml-p15193074p16120542.bz2_red.csv.gz
df_id_alias_title_alias size: (309487, 2)
df_id_alias_id_orig size: (309487, 2)
df_id_orig_title_alias size: (309487, 2)
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_red/enwiki-20180920-pages-meta-current27.xml-p53163462p54663462.bz2_red.csv.gz
df_id_alias_title_alias size: (518951, 2)
df_id_alias_id_orig size: (518951, 2)
df_id_orig_title_alias size: (518951, 2)
/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/art_

In [11]:
df_id_alias_title_alias.head()

,id_alias,title_alias
0,56163490,Shaheedka
1,56163494,Barbarossa cycling trail
2,56163497,Jessica Lange on screen and stage
3,56163499,Barbarossa cycleway
4,56163519,Tomás O'Ryan


In [12]:
df_id_alias_title_alias.to_csv(PATH_TO_DATA+'enwiki-' + data_time + '-id_alias_title_alias.csv', index = False, encoding='UTF-8', quotechar="\"")

In [13]:
df_id_alias_id_orig[['id_orig']] = df_id_alias_id_orig[['id_orig']].fillna(-1).astype(int)

In [14]:
df_id_alias_id_orig.head()

,id_alias,id_orig
0,56163490,954897
1,56163494,56160400
2,56163497,36339503
3,56163499,56160400
4,56163519,20239787


In [15]:
df_id_alias_id_orig.to_csv(PATH_TO_DATA+'enwiki-' + data_time + '-id_alias_id_orig.csv', index = False, encoding='UTF-8', quotechar="\"")

In [16]:
df_id_orig_title_alias[['id_orig']] = df_id_orig_title_alias[['id_orig']].fillna(-1).astype(int)

In [17]:
df_id_orig_title_alias.shape

(8408124, 2)

In [18]:
df_id_orig_title_alias = df_id_orig_title_alias.drop_duplicates()
df_id_orig_title_alias.shape

(8408124, 2)

In [19]:
df_id_orig_title_alias.head()

,id_orig,title_alias
0,954897,Shaheedka
1,56160400,Barbarossa cycling trail
2,36339503,Jessica Lange on screen and stage
3,56160400,Barbarossa cycleway
4,20239787,Tomás O'Ryan


In [20]:
df_id_orig_title_alias.to_csv(PATH_TO_DATA+'enwiki-' + data_time + '-id_orig_title_alias.csv', index = False, encoding='UTF-8', quotechar="\"")